#### Welcome to Session II : Saimese Network 😃!

##### Let's get data and install transformers

In [ ]:
! pip install datasets transformers
# !pip install torch==1.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import pandas as pd
import torch
import os
import pickle
import time
from matplotlib import pyplot as plt

import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AdamW, AutoConfig, get_linear_schedule_with_warmup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### load positive, negative pairs for training and validations 

In [ ]:
similar_train_pairs = pd.read_csv('triplet.csv')

similar_valid_pairs = pd.read_csv('val_triplet.csv')

len(similar_train_pairs)

len(similar_valid_pairs)

In [ ]:
# import numpy as np
# similar_train_pairs.head(2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_anchors = list(similar_train_pairs['anchor'])
train_positives = list(similar_train_pairs['positive'])
train_negatives = list(similar_train_pairs['negative'])
max_length = 35

train_anchor_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in train_anchors]
train_positive_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in train_positives]
train_negative_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in train_negatives]

def get_masks(text_ids):
    att_masks = []
    for ids in text_ids:
        masks = [int(id > 0) for id in ids]
        att_masks.append(masks)
    return att_masks

train_att_masks_anchors = get_masks(train_anchor_ids)
train_att_masks_positives = get_masks(train_positive_ids)  
train_att_masks_negatives = get_masks(train_negative_ids)

t_tanc = torch.tensor(train_anchor_ids)
tm_tanc = torch.tensor(train_att_masks_anchors)
t_tpos = torch.tensor(train_positive_ids)
tm_tpos = torch.tensor(train_att_masks_positives)
t_tneg = torch.tensor(train_negative_ids)
tm_tneg = torch.tensor(train_att_masks_negatives)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
val_anchors = list(similar_valid_pairs['anchor'])
val_positives = list(similar_valid_pairs['positive'])
val_negatives = list(similar_valid_pairs['negative'])
max_length = 35

val_anchor_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in val_anchors]
val_positive_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in val_positives]
val_negative_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in val_negatives]

def get_masks(text_ids):
    att_masks = []
    for ids in text_ids:
        masks = [int(id > 0) for id in ids]
        att_masks.append(masks)
    return att_masks

val_att_masks_anchors = get_masks(val_anchor_ids)
val_att_masks_positives = get_masks(val_positive_ids)  
val_att_masks_negatives = get_masks(val_negative_ids)

v_tanc = torch.tensor(val_anchor_ids)
vm_tanc = torch.tensor(val_att_masks_anchors)
v_tpos = torch.tensor(val_positive_ids)
vm_tpos = torch.tensor(val_att_masks_positives)
v_tneg = torch.tensor(val_negative_ids)
vm_tneg = torch.tensor(val_att_masks_negatives)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

#### create Model 

In [ ]:
config = AutoConfig.from_pretrained('bert-base-uncased', hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.2)
encoder = AutoModel.from_pretrained('bert-base-uncased', config=config)


class Model(torch.nn.Module):
    def __init__(self, encoder, in_dim, out_dim):
        super(Model, self).__init__()
        self.encoder = encoder
        self.fc = torch.nn.Sequential(torch.nn.Linear(in_dim, out_dim), torch.nn.Tanh()) 
        
    def forward(self, x1, m1, x2, m2):
        outputs_1 = self.encoder(x1, m1)
        outputs_2 = self.encoder(x2, m2)
        last_hidden_states_1 = outputs_1.last_hidden_state
        last_hidden_states_2 = outputs_2.last_hidden_state
        sentence_embeddings_1 = torch.mean(last_hidden_states_1, dim=1)
        sentence_embeddings_1 = self.fc(sentence_embeddings_1)
        sentence_embeddings_2 = torch.mean(last_hidden_states_2, dim=1)
        sentence_embeddings_2 = self.fc(sentence_embeddings_2)
        similarity = torch.cosine_similarity(sentence_embeddings_1, sentence_embeddings_2)
        return similarity


model = Model(encoder, 768, 128)
model = model.to(device)


model_loss = torch.nn.MarginRankingLoss(margin=0.3)


adam_epsilon = 1e-8
no_decay = ['bias', 'LayerNorm.weight']


optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.2},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=adam_epsilon)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

### create train and validation DataLoader

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



seed_val = 111
np.random.seed(seed_val)
torch.manual_seed(seed_val)


train_data = TensorDataset(t_tanc, tm_tanc, t_tpos, tm_tpos, t_tneg, tm_tneg)
train_dataloader = DataLoader(train_data, batch_size=100)


val_data = TensorDataset(v_tanc, vm_tanc, v_tpos, vm_tpos, v_tneg, vm_tneg)
val_dataloader = DataLoader(val_data,  batch_size=100)


total_steps = len(train_dataloader) * 100
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
num_mb_train = len(train_dataloader)
num_mb_val = len(val_dataloader)
if num_mb_val == 0:
    num_mb_val = 1

#### Training and Save Model

In [ ]:
# model.load_state_dict(torch.load('similarity_model4.pt'))
# In[ ]:
import time
train_losses = []
val_losses = []

for n in range(8):
    train_loss = 0
    val_loss = 0
    start_time = time.time()
    
    for k, (t_tanc, tm_tanc, t_tpos, tm_tpos, t_tneg, tm_tneg) in enumerate(train_dataloader):
        #print("k=",k,"t_tanc=",t_tanc,"tm_tanc=",tm_tanc)
        optimizer.zero_grad()
        model.train()
        t_tanc = t_tanc.to(device)
        tm_tanc = tm_tanc.to(device)
        t_tpos = t_tpos.to(device)
        tm_tpos = tm_tpos.to(device)
        t_tneg = t_tneg.to(device)
        tm_tneg = tm_tneg.to(device)
        sim_1 = model(t_tanc, tm_tanc, t_tpos, tm_tpos)
        sim_2 = model(t_tanc, tm_tanc, t_tneg, tm_tneg)
        loss = model_loss(sim_1, sim_2, torch.ones_like(sim_1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        train_loss += loss.data / num_mb_train
        #print("train_losses=",train_loss)
    print(len(t_tanc))

    print ('\nTrain loss after iteration %i: %f' % (n+1, train_loss))
    train_losses.append(train_loss.cpu())

    
    with torch.no_grad():
        model.eval()
        for k, (v_tanc, vm_tanc, v_tpos, vm_tpos, v_tneg, vm_tneg) in enumerate(val_dataloader):
            v_tanc = v_tanc.to(device)
            vm_tanc = vm_tanc.to(device)
            v_tpos = v_tpos.to(device)
            vm_tpos = vm_tpos.to(device)
            v_tneg = v_tneg.to(device)
            vm_tneg = vm_tneg.to(device)
            sim_1 = model(v_tanc, vm_tanc, v_tpos, vm_tpos)
            sim_2 = model(v_tanc, vm_tanc, v_tneg, vm_tneg)
            loss = model_loss(sim_1, sim_2, torch.ones_like(sim_1))
            val_loss += loss.data / num_mb_val 
        print ("Validation loss after itaration %i: %f" % (n+1, val_loss))
        val_losses.append(val_loss.cpu())
        
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Time: {epoch_mins}m {epoch_secs}s')
    print(end_time-start_time)

model_name='bert'
triplettype = 'l2'
model_save_path = './similarity_model-'+triplettype+'-'+model_name+'.pt'

torch.save(model.state_dict(),  model_save_path)

train_loss_df  = pd.DataFrame(train_losses)
val_loss_df  = pd.DataFrame(val_losses)
train_loss_df.to_csv("train_loss.csv")
val_loss_df.to_csv("val_loss.csv")

53

Train loss after iteration 1: 0.286331
Validation loss after itaration 1: 0.261489
Time: 12m 39s
759.8412477970123
53

Train loss after iteration 2: 0.274361
Validation loss after itaration 2: 0.256880
Time: 12m 41s
761.62482380867
53

Train loss after iteration 3: 0.265668
Validation loss after itaration 3: 0.257517
Time: 13m 5s
785.2895040512085
53

Train loss after iteration 4: 0.258190
Validation loss after itaration 4: 0.252471
Time: 12m 40s
760.3348052501678
53

Train loss after iteration 5: 0.248732
Validation loss after itaration 5: 0.251322
Time: 12m 9s
729.4995489120483
53

Train loss after iteration 6: 0.241420
Validation loss after itaration 6: 0.253178
Time: 12m 9s
729.8713965415955
53

Train loss after iteration 7: 0.232127
Validation loss after itaration 7: 0.252520
Time: 12m 43s
763.6316256523132
53

Train loss after iteration 8: 0.223270
Validation loss after itaration 8: 0.252807
Time: 12m 14s
734.1253497600555


#### Load saved trained model

In [ ]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'myfolder')
drive.mount(MOUNTPOINT)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
device


device(type='cuda', index=0)

In [ ]:
# !ls "/content/gdrive/My Drive"

In [ ]:
config = AutoConfig.from_pretrained('bert-base-uncased', hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.2)
encoder = AutoModel.from_pretrained('bert-base-uncased', config=config)

model = Model(encoder, 768, 128)
#model.load_state_dict(torch.load("/content/gdrive/My Drive/triplet/similarity_model-l2-bert.pt",map_location=torch.device('cpu')))
model.load_state_dict(torch.load("/content/gdrive/My Drive/triplet/similarity_model-l2-bert.pt"))

# model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

#### Model validation performance 

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset('xglue', 'qadsm')

Reusing dataset xglue (/root/.cache/huggingface/datasets/xglue/qadsm/1.0.0/8566eedecd9ab28e01c051c023dadf97bf408e5195f76b06aba70ebd4697ae08)


  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
val_querys = dataset['validation.en']['query']
val_adtitles = dataset['validation.en']['ad_title']
max_length = 35

val_query_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in val_querys]
val_adtitle_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in val_adtitles]

def get_masks(text_ids):
    att_masks = []
    for ids in text_ids:
        masks = [int(id > 0) for id in ids]
        att_masks.append(masks)
    return att_masks

val_att_masks_querys = get_masks(val_query_ids)
val_att_masks_adtitles = get_masks(val_adtitle_ids)  

val_tanc = torch.tensor(val_query_ids)
valm_tanc = torch.tensor(val_att_masks_querys)
val_title = torch.tensor(val_adtitle_ids)
valm_title = torch.tensor(val_att_masks_adtitles)


In [ ]:
vval_data = TensorDataset(val_tanc, valm_tanc, val_title, valm_title)
vval_dataloader = DataLoader(vval_data, batch_size=100)

In [ ]:
model.eval()
pre_val_sims = []
pre_val_labels =[] 
cnt = 0
start = time.time()
for k, (v_tanc, vm_tanc, v_tpos, vm_tpos) in enumerate(vval_dataloader):
  # v_tanc = v_tanc.to(device)
  # vm_tanc = vm_tanc.to(device)
  # v_tpos = v_tpos.to(device)
  # vm_tpos = vm_tpos.to(device)
  pre_val_sim = model(v_tanc, vm_tanc, v_tpos, vm_tpos)
  pre_val_sims.append(pre_val_sim)
  pre_val_labels.append(pre_val_sim>=0.5)
  cnt = cnt +1 
  print("cnt=",cnt)
  if cnt==1:
    break

time.time() - start

cnt= 1


19.172230005264282

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score


pre_val_list = [ int(i) for i in pre_val_labels[0]]
true_val_list = dataset['test.en']['relevance_label'][0:100]
cm = confusion_matrix(true_val_list, pre_val_list, labels=[0, 1])
tn, fp, fn, tp = confusion_matrix(list(true_val_list), list(pre_val_list), labels=[0, 1]).ravel()

print('validation True Positive', tp)
print('validation True Negative', tn)
print('validation False Positive', fp)
print('validation False Negative', fn)
print("validation accuracy=",accuracy_score(true_val_list, pre_val_list))


print("validation precision=",precision_score(true_val_list, pre_val_list))
print("validation recall=", recall_score(true_val_list, pre_val_list))

validation True Positive 29
validation True Negative 15
validation False Positive 35
validation False Negative 21
validation accuracy= 0.44
validation precision= 0.453125
validation recall= 0.58


### load data , now doing test on test set 

In [ ]:
# ! pip install datasets transformers
# import transformers
# print(transformers.__version__)
# #device
#from datasets import load_dataset, load_metric
#dataset = load_dataset('xglue', 'qadsm')

##### (1) Generate test query and ad_title embedding 
##### (2) Computer similarity 
##### (3) Generate Top-K 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
test_querys = dataset['test.en']['query']
test_adtitles = dataset['test.en']['ad_title']
max_length = 35

test_query_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in test_querys]
test_adtitle_ids = [tokenizer.encode(text, max_length=max_length, truncation=True, padding='max_length') for text in test_adtitles]

def get_masks(text_ids):
    att_masks = []
    for ids in text_ids:
        masks = [int(id > 0) for id in ids]
        att_masks.append(masks)
    return att_masks

test_att_masks_querys = get_masks(test_query_ids)
test_att_masks_adtitles = get_masks(test_adtitle_ids)  

test_tanc = torch.tensor(test_query_ids)
testm_tanc = torch.tensor(test_att_masks_querys)
test_title = torch.tensor(test_adtitle_ids)
testm_title = torch.tensor(test_att_masks_adtitles)


In [ ]:
seed_val = 111
np.random.seed(seed_val)
torch.manual_seed(seed_val)


test_data = TensorDataset(test_tanc, testm_tanc, test_title, testm_title)
test_dataloader = DataLoader(test_data, batch_size=100)#100)

In [ ]:
with torch.no_grad():
  model.eval()
  sim_pos = []
  sim_neg = []
  test_query_embeddings = []
  test_title_embeddings = []
  start = time.time()
  cnt =0 
  for k, (t_tanc, tm_tanc, t_tpos, tm_tpos) in enumerate(test_dataloader):
      t_tanc = t_tanc.to(device)
      tm_tanc = tm_tanc.to(device)
       
      #anchor_embedding = model.fc(torch.mean(model.encoder(t_tanc.cpu(), tm_tanc.cpu()).last_hidden_state, dim=1)).cpu().detach().numpy()
      anchor_embedding = model.fc(torch.mean(model.encoder(t_tanc, tm_tanc).last_hidden_state, dim=1)).cpu().detach().numpy()
      test_query_embeddings.append(anchor_embedding)
      t_tpos = t_tpos.to(device)
      tm_tpos = tm_tpos.to(device)
      pos_embedding = model.fc(torch.mean(model.encoder(t_tpos, tm_tpos).last_hidden_state, dim=1)).cpu().detach().numpy()
      #pos_embedding = model.fc(torch.mean(model.encoder(t_tpos.cpu(), tm_tpos.cpu()).last_hidden_state, dim=1)).cpu().detach().numpy()
      test_title_embeddings.append(pos_embedding)
      cnt =cnt+1
      print("cnt=",cnt)
      if cnt==2:
        break
  test_query_embeddings = np.vstack(test_query_embeddings)
  test_title_embeddings = np.vstack(test_title_embeddings)
  # np.save('test_query_vectors', test_query_embeddings)
  # np.save('test_title_vectors', test_title_embeddings)
  print("test_query_embeddings=",test_query_embeddings)



In [ ]:
pre_test_sims[0]

list

In [ ]:
model.eval()
pre_test_sims = []
pre_test_labels =[] 
cnt = 0
start = time.time()
for k, (t_tanc, tm_tanc, t_tpos, tm_tpos) in enumerate(test_dataloader):
  # t_tanc = t_tanc.to(device)
  # tm_tanc = tm_tanc.to(device)
  # t_tpos = t_tpos.to(device)
  # tm_tpos = tm_tpos.to(device)
  pre_test_sim = model(t_tanc, tm_tanc, t_tpos, tm_tpos)
  pre_test_sims.append(pre_test_sim)
  pre_test_labels.append(pre_test_sim>=0.5)
  cnt = cnt +1 
  if cnt ==1:
    break 
time.time() - start
# print(pre_test_sims)
# print(pre_test_labels)

21.566457271575928

In [ ]:
dataset['test.en'][0:5]

{'ad_description': ['Download & Install Instantly Today. Secure & Live Customer Service!',
  'Over 1MM+ eBooks and bestsellers. Buy Today & Read Anywhere!',
  'Discover Windows 8 Flexibility. Built to Work Easy and Play Hard.',
  "Qualified orders over $25 ship free. Buy 'history Channel' at Amazon!",
  'Discover Windows 8 Flexibility. Built to Work Easy and Play Hard.'],
 'ad_title': ['Office 2010 Now $98',
  'Kindle eBooks at Amazon',
  'Microsoft Windows® 8',
  "'history Channel'",
  'Microsoft Windows® 8'],
 'query': ['free windows office',
  'detroitnews.com',
  'ms security essential update',
  'spontaneous combustion history channel',
  'update windows'],
 'relevance_label': [1, 0, 1, 0, 1]}

In [ ]:
pre_test_labels[0][0:5]

tensor([ True, False, False,  True, False])

In [ ]:
pre_test_sims[0][0:5]

tensor([0.5731, 0.2286, 0.3828, 0.5510, 0.2797], grad_fn=<SliceBackward0>)

In [ ]:
import numpy as np
import pandas as pd
df_test = pd.DataFrame({'pre_sim':pre_test_sims[0].detach().numpy(), 'true_y':pre_test_labels[0].detach().numpy()})
df_test.to_csv("test_pre_sim.csv")

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score


pre_test_list = [ int(i) for i in pre_test_labels[0]]
true_test_list = dataset['test.en']['relevance_label'][0:100]
cm = confusion_matrix(true_test_list, pre_test_list, labels=[0, 1])
tn, fp, fn, tp = confusion_matrix(list(true_test_list), list(pre_test_list), labels=[0, 1]).ravel()

print("accuracy=",accuracy_score(true_test_list, pre_test_list))


print("precision=",precision_score(true_test_list, pre_test_list))
print("recall=", recall_score(true_test_list, pre_test_list))

accuracy= 0.62
precision= 0.6304347826086957
recall= 0.58


#### save embeddings

In [ ]:
#change output vector names here
loadvectors = False
if loadvectors:
    test_embeddings = np.load('output/test_qvectors.npy')
    train_embeddings = np.load('output/test_tvectors.npy')
else:
    model.eval()
    sim_pos = []
    sim_neg = []
    test_query_embeddings = []
    test_title_embeddings = []
    start = time.time()
    for k, (t_tanc, tm_tanc, t_tpos, tm_tpos) in enumerate(test_dataloader):
        # t_tanc = t_tanc.to(device)
        # tm_tanc = tm_tanc.to(device)
        q_embedding = model.fc(torch.mean(model.encoder(t_tanc, tm_tanc).last_hidden_state, dim=1)).cpu().detach().numpy()
        test_query_embeddings.append(q_embedding)
        t_embedding = model.fc(torch.mean(model.encoder(t_tpos, tm_tpos).last_hidden_state, dim=1)).cpu().detach().numpy()
        test_title_embeddings.append(t_embedding)
        break

    test_query_embeddings = np.vstack(q_embedding)
    test_title_embeddings = np.vstack(t_embedding)
    np.save('test_qvectors', test_query_embeddings)
    np.save('test_tvectors', test_title_embeddings)
    time.time() - start

In [ ]:
# model.eval()
# pre_pos_sims = []
# pre_neg_sims = []
# train_embeddings = []
# start = time.time()
# for k, (v_tanc, vm_tanc, v_tpos, vm_tpos, v_tneg, vm_tneg) in enumerate(val_dataloader):
#   v_tanc = v_tanc.to(device)
#   vm_tanc = vm_tanc.to(device)
#   v_tpos = v_tpos.to(device)
#   vm_tpos = vm_tpos.to(device)
#   v_tneg = v_tneg.to(device)
#   vm_tneg = vm_tneg.to(device)
#   pre_pos_sim = model(v_tanc, vm_tanc, v_tpos, vm_tpos)
#   pre_neg_sim = model(v_tanc, vm_tanc, v_tneg, vm_tneg)
#   pre_pos_sims.append(pre_pos_sim)
#   pre_neg_sims.append(pre_neg_sim)
  
# time.time() - start

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
test_q_embeddings = np.load('./test_qvectors.npy')
test_t_embeddings = np.load('./test_tvectors.npy')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
X = np.array([[1,2],[2,5]])
Y = np.array([[2,2]])
Z = np.array([2,4])

# calculate cosine similarity between [X] and [Y,Z]
# sending input as arrays would allow for calculating both cosine_sim(X,Y) and cosine_sim (X,Y)
print("X.shape=",X.shape, "Y.shape=",Y.shape)
cos_sim = cosine_similarity(X, Y)
print(cos_sim)
print(cos_sim.shape)
np.argsort(-cos_sim, axis=1)#[0:2]

X.shape= (2, 2) Y.shape= (1, 2)
[[0.9486833 ]
 [0.91914503]]
(2, 1)


array([[0],
       [0]])

In [ ]:
similarity = 0
indexes = 0
score = 0
batch_size = 100
batches = test_q_embeddings.shape[0] // batch_size
print("batches=",batches)

if test_q_embeddings.shape[0]%batch_size != 0:
    batches = batches + 1 

for row_i in range(0, batches):
    similarity = 0
    indexes = 0
    score = 0
    start = row_i * batch_size
    end = min([(row_i + 1) * batch_size, test_q_embeddings.shape[0]])        
    rows = test_q_embeddings[start: end]
    print("no. rows=",rows.shape,"test_t_embeddings.shape=",test_t_embeddings.shape)
    similarity = cosine_similarity(rows, test_t_embeddings)  
    print("similarity.shape=",similarity.shape)
    ## top-1 
    indexes = np.argmax(similarity, axis=1)
    print("top-1 indexes=",indexes)
    ## top-3 
    # print(np.argsort(similarity, axis=1)[:,-3:])
    # score = np.max(similarity, axis=1)
    # print("len(indexes)=",len(indexes),"len(score)=",len(score))
    # print("indexes=",indexes,"score=",score)
    break

batches= 1
no. rows= (100, 128) test_t_embeddings.shape= (100, 128)
similarity.shape= (100, 100)
top-1 indexes= [ 0  0 74  3 74 51  6 27  6  9 74 74 74 59 14 15  1 48  3 51 94 21 22 23
 24 51 18 27 28 54 30 21 66 33 66  0 88 71 46 51 40 78 42 43 42 88 53 47
 22 49 50 51 52 53 57 22 63 28 58 59 60 19 60 63 64 23 66 86 27 69 70  0
 72  7 74 72 76 77 96 77 80 77 80 79 94 26 80 26 88 26 90 26 92 26 96 26
 96 16 96 18]


In [ ]:
indexes[0:5]

# dataset['test.en'][0]

array([ 0,  0, 74,  3, 74])

In [ ]:
score[0:5]

array([0.5731111 , 0.45155165, 0.57501847, 0.5510431 , 0.3222438 ],
      dtype=float32)

In [ ]:
dataset['test.en'][0]

{'ad_description': 'Download & Install Instantly Today. Secure & Live Customer Service!',
 'ad_title': 'Office 2010 Now $98',
 'query': 'free windows office',
 'relevance_label': 1}

In [ ]:
dataset['test.en'][1]

{'ad_description': 'Over 1MM+ eBooks and bestsellers. Buy Today & Read Anywhere!',
 'ad_title': 'Kindle eBooks at Amazon',
 'query': 'detroitnews.com',
 'relevance_label': 0}